In [2]:
import pandas as pd
import pyodbc

In [3]:
df = pd.read_csv('for_isertion_df.csv')
df.head()

,Company,Job Title,Location,Job Type,Experience level,Requirment of the company,Facilities,AVG_Salary,Negotiation,asterisk,Country,job role
0,SGS,clinical data analyst,"['richardson', 'tx', 'united states']",Full Time,Entry-level,"['computer science', 'data quality', 'genetics...",No Facilities,48000.0,True,True,united states,data Analyste
1,Ocorian,aml/cft & data analyst,"['ebne', 'mauritius']",Full Time,Entry-level,"['agile', 'data management', 'finance', 'secur...",No Facilities,48000.0,True,True,mauritius,data Analyste
2,Cricut,machine learning engineer,"['south jordan', 'ut', 'united states']",Full Time,Not-Specified,"['agile', 'architecture', 'aws', 'computer sci...",['Career development'],90000.0,True,True,united states,IA
3,Bosch Group,application developer & data analyst,"['nonantola', 'italy']",Full Time,Entry-level,"['engineering', 'industrial', 'oracle', 'power...",No Facilities,48000.0,True,True,italy,data Analyste
4,Publicis Groupe,data engineer full time (public sector) usa,"['arlington', 'va', 'united states']",Full Time,Mid-level,"['aws', 'azure', 'computer science', 'consulti...","['Flex hours', 'Flex vacation', 'Parental leav...",108000.0,True,False,united states,data engineer


In [6]:
df['job role'].value_counts().index

Index(['data scientist', 'data engineer', 'data Analyste', 'IA', 'big data',
       'other'],
      dtype='object')

In [ ]:
# Créer un connexion dans le database :
driver = 'SQL Server'
server = 'LAPTOP-6D8J0VI4\SQLEXPRESS'
database = 'github_DB'

conn = pyodbc.connect(f'''
                        DRIVER={driver};
                        SERVER={server};
                        DATABASE={database};
                        Trusted_Connection=yes
                       ''')
cursor = conn.cursor()
conn

In [10]:
cursor.execute('''
               CREATE TABLE [job_opportunity] (
                [Id] INT NOT NULL,
                [Company_id] INT NOT NULL,
                [Location_id] INT NOT NULL,
                [Country_id] INT NOT NULL,
                [Job_Titles_id] INT NOT NULL,
                [Job_Type] VARCHAR(50) NOT NULL,
                [Experience_level] VARCHAR(50) NOT NULL,
                [AVG_Salary] FLOAT NOT NULL,
                [Negotiation] BIT NOT NULL,
                [custom_asterisk] BIT NOT NULL,
                [job_role] VARCHAR(50) NOT NULL,
                CONSTRAINT [PK_job_opportunity] PRIMARY KEY CLUSTERED ([Id] ASC)
            )

            CREATE TABLE [Company] (
                [Id] INT NOT NULL,
                [Name] VARCHAR(100) NOT NULL,
                CONSTRAINT [PK_Company] PRIMARY KEY CLUSTERED ([Id] ASC)
            )

            CREATE TABLE [Location] (
                [Id] INT NOT NULL,
                [Full_location] VARCHAR(100) NOT NULL,
                CONSTRAINT [PK_Location] PRIMARY KEY CLUSTERED ([Id] ASC)
            )

            CREATE TABLE [Country] (
                [id] INT NOT NULL,
                [Country_name] VARCHAR(100) NOT NULL,
                CONSTRAINT [PK_Country] PRIMARY KEY CLUSTERED ([id] ASC)
            )

            CREATE TABLE [Requirment_of_the_company] (
                [Id] INT NOT NULL,
                [Name] VARCHAR(1000) NOT NULL,
                CONSTRAINT [PK_Requirment_of_the_company] PRIMARY KEY CLUSTERED ([Id] ASC)
            )

            CREATE TABLE [Facilities] (
                [Id] INT NOT NULL,
                [Name] VARCHAR(1000) NOT NULL,
                CONSTRAINT [PK_Facilities] PRIMARY KEY CLUSTERED ([Id] ASC)
            )

            CREATE TABLE [Job_Titles] (
                [Id] INT NOT NULL,
                [Name] VARCHAR(1000) NOT NULL,
                CONSTRAINT [PK_Job_Titles] PRIMARY KEY CLUSTERED ([Id] ASC)
            )

            CREATE TABLE [Requirment_bridge] (
                [job_opportunity_id] INT NOT NULL,
                [Requirment_of_the_company_id] INT NOT NULL,
                CONSTRAINT [FK_Requirment_bridge_job_opportunity_id] FOREIGN KEY([job_opportunity_id])
                REFERENCES [job_opportunity] ([Id])
            )

            CREATE TABLE [Facilities_of_the_job] (
                [job_opportunity_id] INT NOT NULL,
                [Facility_id] INT NOT NULL,
                CONSTRAINT [FK_Facilities_of_the_job_job_opportunity_id] FOREIGN KEY([job_opportunity_id])
                REFERENCES [job_opportunity] ([Id])
            )

            ALTER TABLE [job_opportunity] WITH CHECK ADD CONSTRAINT [FK_job_opportunity_Company_id] FOREIGN KEY([Company_id])
            REFERENCES [Company] ([Id])

            ALTER TABLE [job_opportunity] CHECK CONSTRAINT [FK_job_opportunity_Company_id]

            ALTER TABLE [job_opportunity] WITH CHECK ADD CONSTRAINT [FK_job_opportunity_Location_id] FOREIGN KEY([Location_id])
            REFERENCES [Location] ([Id])

            ALTER TABLE [job_opportunity] CHECK CONSTRAINT [FK_job_opportunity_Location_id]

            ALTER TABLE [job_opportunity] WITH CHECK ADD CONSTRAINT [FK_job_opportunity_Country_id] FOREIGN KEY([Country_id])
            REFERENCES [Country] ([id])

            ALTER TABLE [job_opportunity] CHECK CONSTRAINT [FK_job_opportunity_Country_id]

            ALTER TABLE [job_opportunity] WITH CHECK ADD CONSTRAINT [FK_job_opportunity_Job_Titles_id] FOREIGN KEY([Job_Titles_id])
            REFERENCES [Job_Titles] ([Id])

            ALTER TABLE [job_opportunity] CHECK CONSTRAINT [FK_job_opportunity_Job_Titles_id]

            ALTER TABLE [Requirment_bridge] WITH CHECK ADD CONSTRAINT [FK_Requirment_bridge_Requirment_of_the_company_id] FOREIGN KEY([Requirment_of_the_company_id])
            REFERENCES [Requirment_of_the_company] ([Id])

            ALTER TABLE [Requirment_bridge] CHECK CONSTRAINT [FK_Requirment_bridge_Requirment_of_the_company_id]

            ALTER TABLE [Facilities_of_the_job] WITH CHECK ADD CONSTRAINT [FK_Facilities_of_the_job_Facility_id] FOREIGN KEY([Facility_id])
            REFERENCES [Facilities] ([Id])

            ALTER TABLE [Facilities_of_the_job] CHECK CONSTRAINT [FK_Facilities_of_the_job_Facility_id]

               );''')

NameError: name 'cursor' is not defined